In [7]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

warnings.filterwarnings('ignore')

In [8]:
# :seed to make all processes deterministic     # type: int
def seed_everything(seed=0):
    random.seed(seed)
    np.random.seed(seed)

In [9]:
# Read data
def get_data_by_store(store):
    
    # Read and contact basic feature
    df = pd.concat([pd.read_pickle(BASE),
                    pd.read_pickle(PRICE).iloc[:,2:],
                    pd.read_pickle(CALENDAR).iloc[:,2:]],
                    axis=1)
    
    # Leave only relevant store
    df = df[df['store_id']==store]

    # With memory limits we have to read 
    # lags and mean encoding features
    # separately and drop items that we don't need.
    # As our Features Grids are aligned 
    # we can use index to keep only necessary rows
    # Alignment is good for us as concat uses less memory than merge.
    df2 = pd.read_pickle(MEAN_ENC)[mean_features]
    df2 = df2[df2.index.isin(df.index)]
    
    df3 = pd.read_pickle(LAGS).iloc[:,3:]
    df3 = df3[df3.index.isin(df.index)]
    
    df = pd.concat([df, df2], axis=1)
    del df2 # to not reach memory limit 
    
    df = pd.concat([df, df3], axis=1)
    del df3 # to not reach memory limit 
    
    if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster','snow_m']
    else:
        remove_features = ['id','state_id','store_id','date','wm_yr_wk','d',TARGET,'cluster']
    
    # Create features list
    features = [col for col in list(df) if col not in remove_features]
    df = df[['id','d',TARGET]+features]
    
    # Skipping first n rows
    df = df[df['d']>=START_TRAIN].reset_index(drop=True)
    
    return df, features

# Recombine Test set after training
def get_base_test():
    base_test = pd.DataFrame()

    for store_id in STORES_IDS:
        temp_df = pd.read_pickle('test_'+store_id+str(VER)+'.pkl')
        temp_df['store_id'] = store_id
        base_test = pd.concat([base_test, temp_df]).reset_index(drop=True)
    
    return base_test


########################### Helper to make dynamic rolling lags
#################################################################################
def make_lag(LAG_DAY):
    lag_df = base_test[['id','d',TARGET]]
    col_name = 'sales_lag_'+str(LAG_DAY)
    lag_df[col_name] = lag_df.groupby(['id'])[TARGET].transform(lambda x: x.shift(LAG_DAY)).astype(np.float16)
    return lag_df[[col_name]]

In [10]:
def make_lag_roll(LAG_DAY,lag_df_new):
   
    lag_df = base_test[['id','d',TARGET]]
    
    lag_df=lag_df.sort_values(by=["d"])
  
    for i in range(0,len(LAG_DAY)):

        shift_day = LAG_DAY[i][0]
        roll_wind = LAG_DAY[i][1]
        col_name = 'rolling_mean_tmp_'+str(shift_day)+'_'+str(roll_wind)
        lag_df[col_name] = (lag_df.groupby(['id'])[TARGET]).transform(lambda x: x.shift(shift_day).rolling(roll_wind).mean())
    lag_df_new=lag_df.drop(columns=["sales"])
    return lag_df_new

In [11]:
import lightgbm as lgb
lgb_params = {
                    'boosting_type': 'gbdt',
                    'objective': 'tweedie',
                    'tweedie_variance_power': 1.1,
                    'metric': 'rmse',
                    'subsample': 0.5,
                    'subsample_freq': 1,
                    'learning_rate': 0.03,
                    "lambda":0.1,
                    'num_leaves': 2**11-1,
                    'min_data_in_leaf': 2**12-1,
                    'feature_fraction': 0.5,
                    'max_bin': 100,
                    'n_estimators': 1400,
                    'boost_from_average': False,
                    'verbose': -1,
                } 



# lgb_params ={
#         "objective" : "tweedie",
#         "metric" :"rmse",
#         "force_row_wise" : True,
#         "learning_rate" : 0.075,
#         "sub_feature" : 0.8,
#         "sub_row" : 0.75,
#         "bagging_freq" : 1,
#         "lambda_l2" : 0.1,
#         "metric": ["rmse"],
#         "nthread": -1,
#         "tweedie_variance_power":1.1,
#     'verbosity': 1,
# #     'num_iterations' : 1500,
#     'num_leaves': 128,
#     "min_data_in_leaf": 104,
#     }




# Let's look closer on params

## 'boosting_type': 'gbdt'
# we have 'goss' option for faster training
# but it normally leads to underfit.
# Also there is good 'dart' mode
# but it takes forever to train
# and model performance depends 
# a lot on random factor 
# https://www.kaggle.com/c/home-credit-default-risk/discussion/60921

## 'objective': 'tweedie'
# Tweedie Gradient Boosting for Extremely
# Unbalanced Zero-inflated Data
# https://arxiv.org/pdf/1811.10192.pdf
# and many more articles about tweediie
#
# Strange (for me) but Tweedie is close in results
# to my own ugly loss.
# My advice here - make OWN LOSS function
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/140564
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/143070
# I think many of you already using it (after poisson kernel appeared) 
# (kagglers are very good with "params" testing and tuning).
# Try to figure out why Tweedie works.
# probably it will show you new features options
# or data transformation (Target transformation?).

## 'tweedie_variance_power': 1.1
# default = 1.5
# set this closer to 2 to shift towards a Gamma distribution
# set this closer to 1 to shift towards a Poisson distribution
# my CV shows 1.1 is optimal 
# but you can make your own choice

## 'metric': 'rmse'
# Doesn't mean anything to us
# as competition metric is different
# and we don't use early stoppings here.
# So rmse serves just for general 
# model performance overview.
# Also we use "fake" validation set
# (as it makes part of the training set)
# so even general rmse score doesn't mean anything))
# https://www.kaggle.com/c/m5-forecasting-accuracy/discussion/133834

## 'subsample': 0.5
# Serves to fight with overfit
# this will randomly select part of data without resampling
# Chosen by CV (my CV can be wrong!)
# Next kernel will be about CV

##'subsample_freq': 1
# frequency for bagging
# default value - seems ok

## 'learning_rate': 0.03
# Chosen by CV
# Smaller - longer training
# but there is an option to stop 
# in "local minimum"
# Bigger - faster training
# but there is a chance to
# not find "global minimum" minimum

## 'num_leaves': 2**11-1
## 'min_data_in_leaf': 2**12-1
# Force model to use more features
# We need it to reduce "recursive"
# error impact.
# Also it leads to overfit
# that's why we use small 

# 'max_bin': 100
## l1, l2 regularizations
# https://towardsdatascience.com/l1-and-l2-regularization-methods-ce25e7fc831c
# Good tiny explanation
# l2 can work with bigger num_leaves
# but my CV doesn't show boost
                    
## 'n_estimators': 1400
# CV shows that there should be
# different values for each state/store.
# Current value was chosen 
# for general purpose.
# As we don't use any early stopings
# careful to not overfit Public LB.

##'feature_fraction': 0.5
# LightGBM will randomly select 
# part of features on each iteration (tree).
# We have maaaany features
# and many of them are "duplicates"
# and many just "noise"
# good values here - 0.5-0.7 (by CV)

## 'boost_from_average': False
# There is some "problem"
# to code boost_from_average for 
# custom loss
# 'True' makes training faster
# BUT carefull use it
# https://github.com/microsoft/LightGBM/issues/1514

In [12]:
VER = 7                       # Our model version
SEED = 42                      # We want all things
seed_everything(SEED)            # to be as deterministic 
lgb_params['seed'] = SEED        # as possible
N_CORES = psutil.cpu_count()     # Available CPU cores


#LIMITS and const
TARGET      = 'sales'            # Our target
START_TRAIN = 0                  # We can skip some rows (Nans/faster training)
END_TRAIN   = 1941               # End day of our train set, change this part for final
P_HORIZON   = 28                 # Prediction horizon

#FEATURES to remove
## These features lead to overfit
## or values not present in test set
mean_features   = ['enc_cat_id_mean','enc_cat_id_std',
                   'enc_dept_id_mean','enc_dept_id_std',
                   'enc_item_id_mean','enc_item_id_std'] 

#PATHS for Features
BASE     = 'grid_part_1.pkl'
PRICE    = 'grid_part_2.pkl'
CALENDAR = 'grid_part_3.pkl'
LAGS     = 'lags_df_28_v3.pkl'
MEAN_ENC = 'mean_encoding_df.pkl'


# AUX(pretrained) Models paths

#STORES ids
STORES_IDS = pd.read_csv('sales_train_evaluation.csv')['store_id']#change this part for final
STORES_IDS = list(STORES_IDS.unique())

#SPLITS for lags creation
SHIFT_DAY  = 28
N_LAGS     = 15
LAGS_SPLIT = [col for col in range(SHIFT_DAY,SHIFT_DAY+N_LAGS)]
ROLS_SPLIT = []
for i in [1,7,14]:
    for j in [7,14,28,56]:
        ROLS_SPLIT.append([i,j])

In [13]:
for store_id in STORES_IDS:
    print('Train', store_id)
    
    # Get grid for current store
    grid_df, features_columns = get_data_by_store(store_id)
    
    print(features_columns)
    # Masks for 
    # Train (All data less than 1913)
    # "Validation" (Last 28 days - not real validation set)
    # Test (All data greater than 1913 day, 
    #       with some gap for recursive features)
    train_mask = grid_df['d']<=END_TRAIN
    valid_mask = train_mask&(grid_df['d']>(END_TRAIN-P_HORIZON))
    preds_mask = grid_df['d']>(END_TRAIN-100)
    
    # Apply masks and save lgb dataset as bin
    # to reduce memory spikes during dtype convertations
    # https://github.com/Microsoft/LightGBM/issues/1032
    # "To avoid any conversions, you should always use np.float32"
    # or save to bin before start training
    # https://www.kaggle.com/c/talkingdata-adtracking-fraud-detection/discussion/53773
    train_data = lgb.Dataset(grid_df[train_mask][features_columns], 
                       label=grid_df[train_mask][TARGET])
    
    
    valid_data = lgb.Dataset(grid_df[valid_mask][features_columns], 
                       label=grid_df[valid_mask][TARGET])
    
    # Saving part of the dataset for later predictions
    # Removing features that we need to calculate recursively 
    grid_df = grid_df[preds_mask].reset_index(drop=True)
    keep_cols = [col for col in list(grid_df) if '_tmp_' not in col]
    grid_df = grid_df[keep_cols]
    grid_df.to_pickle('test_'+store_id+str(VER)+'.pkl')
    del grid_df
    gc.collect()
    
    # Launch seeder again to make lgb training 100% deterministic
    # with each "code line" np.random "evolves" 
    # so we need (may want) to "reset" it
    seed_everything(SEED)
    estimator = lgb.train(lgb_params,
                          train_data,
                          valid_sets = [valid_data],
                          verbose_eval = 100,
                          )
    imp_type = "gain"
    features = estimator.feature_name()
    importances = estimator.feature_importance(imp_type)
    importance_df=pd.DataFrame(features,columns=['features'])
    importance_df['importances']=importances
    importance_df=importance_df.sort_values(by='importances', ascending=False)
    importance_df.to_csv(store_id+'_fe_imp_'+str(VER)+'.csv',index=False)
    del importance_df
    gc.collect()
    
    # Save model - it's not real '.bin' but a pickle file
    # estimator = lgb.Booster(model_file='model.txt')
    # can only predict with the best iteration (or the saving iteration)
    # pickle.dump gives us more flexibility
    # like estimator.predict(TEST, num_iteration=100)
    # num_iteration - number of iteration want to predict with, 
    # NULL or <= 0 means use best iteration
    model_name = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin'
    pickle.dump(estimator, open(model_name, 'wb'))

    # Remove temporary files and objects 
    # to free some hdd space and ram memory
    #     !rm train_data.bin
    del train_data, valid_data, estimator
    gc.collect()

Train CA_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean', 'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std', 'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39', 'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7',

[100]	valid_0's rmse: 1.39236
[200]	valid_0's rmse: 1.38348
[300]	valid_0's rmse: 1.37813
[400]	valid_0's rmse: 1.37388
[500]	valid_0's rmse: 1.3699
[600]	valid_0's rmse: 1.36648
[700]	valid_0's rmse: 1.36296
[800]	valid_0's rmse: 1.35934
[900]	valid_0's rmse: 1.35605
[1000]	valid_0's rmse: 1.35288
[1100]	valid_0's rmse: 1.35008
[1200]	valid_0's rmse: 1.34721
[1300]	valid_0's rmse: 1.34445
[1400]	valid_0's rmse: 1.34178
Train TX_1
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm

[100]	valid_0's rmse: 1.60689
[200]	valid_0's rmse: 1.58277
[300]	valid_0's rmse: 1.57418
[400]	valid_0's rmse: 1.56801
[500]	valid_0's rmse: 1.56318
[600]	valid_0's rmse: 1.55865
[700]	valid_0's rmse: 1.55434
[800]	valid_0's rmse: 1.55015
[900]	valid_0's rmse: 1.54614
[1000]	valid_0's rmse: 1.54259
[1100]	valid_0's rmse: 1.53886
[1200]	valid_0's rmse: 1.5352
[1300]	valid_0's rmse: 1.53166
[1400]	valid_0's rmse: 1.52843
Train WI_2
['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min', 'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 'tm_d', 'tm_w', 'tm_m', '

In [14]:
# Create Dummy DataFrame to store predictions
all_preds = pd.DataFrame()

# Join back the Test dataset with 
# a small part of the training data 
# to make recursive features
base_test = get_base_test()

# Timer to measure predictions time 
main_time = time.time()

# Loop over each prediction day
# As rolling lags are the most timeconsuming
# we will calculate it for whole day


for PREDICT_DAY in range(1,29):    
    print('Predict | Day:', PREDICT_DAY)
    start_time = time.time()

    # Make temporary grid to calculate rolling lags
    grid_df = base_test.copy()
    
    
    lag_df_new = pd.DataFrame()

    lag_df_new=make_lag_roll(ROLS_SPLIT,lag_df_new)


    grid_df = grid_df.merge(lag_df_new, on=['id','d'], how='left')


    for store_id in STORES_IDS:
        
        if store_id in ['CA_1', 'CA_2', 'CA_3','CA_4','TX_1','TX_2','TX_3']:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min',
                              'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 
                              'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 
                              'temperature_high', 'temperature_con', 'rainfall_m', 'event_name_1', 
                              'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 
                              'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 
                              'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean',
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std',
                              'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 
                              'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39',
                              'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 
                              'rolling_std_14', 'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 'rolling_mean_168',
                              'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 
                              'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56',
                              'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 
                              'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 
                              'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56',
                              'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 
                              'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']
        else:
            MODEL_FEATURES = ['item_id', 'dept_id', 'cat_id', 'release', 'sell_price', 'price_max', 'price_min',
                              'price_std', 'price_mean', 'price_norm', 'price_rank_dept', 'price_nunique', 
                              'item_nunique', 'price_momentum', 'price_momentum_m', 'price_momentum_y', 
                              'temperature_high', 'temperature_con', 'rainfall_m', 'snow_m','event_name_1', 
                              'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 
                              'snap_WI', 'is_first_half_month', 'event_bef_weekend', 'event_after_weekend', 
                              'NBA', 'event_attention_after', 'event_attention_bef', 'event_attention_sum', 
                              'tm_d', 'tm_w', 'tm_m', 'tm_q', 'tm_y', 'tm_wm', 'tm_dw', 'tm_w_end', 'enc_cat_id_mean',
                              'enc_cat_id_std', 'enc_dept_id_mean', 'enc_dept_id_std', 'enc_item_id_mean', 'enc_item_id_std',
                              'sales_lag_28', 'sales_lag_29', 'sales_lag_30', 'sales_lag_31', 'sales_lag_32', 'sales_lag_33', 
                              'sales_lag_34', 'sales_lag_35', 'sales_lag_36', 'sales_lag_37', 'sales_lag_38', 'sales_lag_39',
                              'sales_lag_40', 'sales_lag_41', 'sales_lag_42', 'rolling_mean_7', 'rolling_std_7', 'rolling_mean_14', 
                              'rolling_std_14', 'rolling_mean_28', 'rolling_std_28', 'rolling_mean_56', 'rolling_std_56', 'rolling_mean_168',
                              'rolling_std_168', 'rolling_mean_tmp_1_7', 'rolling_mean_tmp_1_14', 'rolling_mean_tmp_1_28', 'rolling_mean_tmp_1_56', 
                              'rolling_mean_tmp_7_7', 'rolling_mean_tmp_7_14', 'rolling_mean_tmp_7_28', 'rolling_mean_tmp_7_56', 
                              'rolling_mean_tmp_14_7', 'rolling_mean_tmp_14_14', 'rolling_mean_tmp_14_28', 'rolling_mean_tmp_14_56',
                              'rolling_quantile_97_28', 'rolling_quantile_87.5_28', 'rolling_quantile_50_28', 
                              'rolling_quantile_22.5_28', 'rolling_quantile_3_28', 'rolling_quantile_97_56', 
                              'rolling_quantile_87.5_56', 'rolling_quantile_50_56', 'rolling_quantile_22.5_56',
                              'rolling_quantile_3_56', 'rolling_quantile_97_168', 'rolling_quantile_87.5_168', 
                              'rolling_quantile_50_168', 'rolling_quantile_22.5_168', 'rolling_quantile_3_168']

        # Read all our models and make predictions
        # for each day/store pairs
        model_path = 'lgb_model_'+store_id+'_v'+str(VER)+'.bin' 

        estimator = pickle.load(open(model_path, 'rb'))

        day_mask = base_test['d']==(END_TRAIN+PREDICT_DAY)
        store_mask = base_test['store_id']==store_id

        mask = (day_mask)&(store_mask)
        base_test[TARGET][mask] = estimator.predict(grid_df[mask][MODEL_FEATURES])

    # Make good column naming and add 
    # to all_preds DataFrame
    temp_df = base_test[day_mask][['id',TARGET]]
    temp_df.columns = ['id','F'+str(PREDICT_DAY)]
    if 'id' in list(all_preds):
        all_preds = all_preds.merge(temp_df, on=['id'], how='left')
    else:
        all_preds = temp_df.copy()

    print('#'*10, ' %0.2f min round |' % ((time.time() - start_time) / 60),
                  ' %0.2f min total |' % ((time.time() - main_time) / 60),
                  ' %0.2f day sales |' % (temp_df['F'+str(PREDICT_DAY)].sum()))
    
    del temp_df, lag_df_new

all_preds = all_preds.reset_index(drop=True)
all_preds.head()

Predict | Day: 1
##########  5.09 min round |  5.09 min total |  39994.80 day sales |
Predict | Day: 2
##########  5.27 min round |  10.35 min total |  37474.79 day sales |
Predict | Day: 3
##########  5.34 min round |  15.70 min total |  37025.19 day sales |
Predict | Day: 4
##########  5.06 min round |  20.75 min total |  37288.91 day sales |
Predict | Day: 5
##########  5.03 min round |  25.78 min total |  42811.64 day sales |
Predict | Day: 6
##########  5.02 min round |  30.80 min total |  50716.83 day sales |
Predict | Day: 7
##########  5.01 min round |  35.81 min total |  51343.54 day sales |
Predict | Day: 8
##########  5.01 min round |  40.82 min total |  45544.07 day sales |
Predict | Day: 9
##########  4.98 min round |  45.81 min total |  39022.93 day sales |
Predict | Day: 10
##########  5.25 min round |  51.06 min total |  44037.07 day sales |
Predict | Day: 11
##########  5.26 min round |  56.31 min total |  45627.60 day sales |
Predict | Day: 12
##########  5.29 min rou

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_evaluation,0.894650,0.740824,0.743733,0.802512,1.090515,1.252276,1.107341,0.969245,0.840303,...,0.973964,1.343931,1.201659,0.995258,0.871385,0.766582,0.839343,1.050457,1.134294,1.052629
1,HOBBIES_1_002_CA_1_evaluation,0.213923,0.191861,0.173980,0.181938,0.208230,0.283084,0.316476,0.238132,0.188781,...,0.256312,0.347537,0.384805,0.233512,0.238856,0.240991,0.237374,0.290065,0.411067,0.399632
2,HOBBIES_1_003_CA_1_evaluation,0.515465,0.459745,0.500881,0.502813,0.623014,0.780394,0.801675,0.487788,0.441010,...,0.551392,0.669063,0.735194,0.461973,0.424064,0.449492,0.477225,0.660238,0.703577,0.704070
3,HOBBIES_1_004_CA_1_evaluation,1.581147,1.224077,1.269111,1.305957,1.856952,2.586149,2.846495,2.000127,1.462243,...,1.871365,2.541568,3.039908,1.720803,1.349174,1.370145,1.449958,1.948558,2.599942,2.776232
4,HOBBIES_1_005_CA_1_evaluation,1.106617,0.909783,0.908104,0.928132,1.195041,1.374480,1.449861,1.166438,1.003477,...,1.221315,1.457311,1.495439,1.098608,1.031743,1.041607,0.993719,1.334914,1.643681,1.424295


In [15]:
# all the following is changed
# replace validation part
train_df = pd.read_csv('sales_train_evaluation.csv')
train_df=train_df[['id','d_1914','d_1915','d_1916','d_1917','d_1918','d_1919','d_1920','d_1921','d_1922','d_1923',
                  'd_1924','d_1925','d_1926','d_1927','d_1928','d_1929','d_1930','d_1931','d_1932','d_1933',
                  'd_1934','d_1935','d_1936','d_1937','d_1938','d_1939','d_1940','d_1941']]

In [16]:
submission = pd.read_csv('sample_submission.csv')

In [17]:
train_df['id']=train_df['id'].str.replace('evaluation','validation')

In [18]:
train_df.columns=submission.columns

In [19]:
submission = submission[['id']]
sub1 = submission.merge(train_df, on=['id'], how='left')

In [20]:
sub1=sub1[:30490]

In [21]:
sub2 = submission.merge(all_preds, on=['id'], how='left')

In [22]:
sub2=sub2[30490:]

In [23]:
final_sub=pd.concat([sub1,sub2],axis=0)

In [24]:
final_sub.head()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,0.0,0.0,0.0,2.0,0.0,3.0,5.0,0.0,0.0,...,2.0,4.0,0.0,0.0,0.0,0.0,3.0,3.0,0.0,1.0
1,HOBBIES_1_002_CA_1_validation,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,2.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0
2,HOBBIES_1_003_CA_1_validation,0.0,0.0,1.0,1.0,0.0,2.0,1.0,0.0,0.0,...,1.0,0.0,2.0,0.0,0.0,0.0,2.0,3.0,0.0,1.0
3,HOBBIES_1_004_CA_1_validation,0.0,0.0,1.0,2.0,4.0,1.0,6.0,4.0,0.0,...,1.0,1.0,0.0,4.0,0.0,1.0,3.0,0.0,2.0,6.0
4,HOBBIES_1_005_CA_1_validation,1.0,0.0,2.0,3.0,1.0,0.0,3.0,2.0,3.0,...,0.0,0.0,0.0,2.0,1.0,0.0,0.0,2.0,1.0,0.0


In [25]:
final_sub.tail()

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
60975,FOODS_3_823_WI_3_evaluation,0.490337,0.465631,0.482985,0.475922,0.539167,0.552109,0.650007,0.519350,0.488419,...,0.583475,0.808103,0.918172,0.555415,0.584312,0.634814,0.539123,0.502260,0.599267,0.677082
60976,FOODS_3_824_WI_3_evaluation,0.268979,0.237821,0.257196,0.248251,0.236453,0.296714,0.352478,0.295741,0.276441,...,0.281340,0.410312,0.439606,0.296148,0.354616,0.388308,0.293266,0.258052,0.311611,0.381592
60977,FOODS_3_825_WI_3_evaluation,0.669850,0.501254,0.471308,0.440993,0.549240,0.608556,0.654940,0.706753,0.548927,...,0.865902,1.301450,1.305785,1.029473,1.056286,1.017565,0.715052,0.759433,0.798392,0.945131
60978,FOODS_3_826_WI_3_evaluation,1.097500,1.115905,0.988102,1.049543,1.178411,1.214723,1.207105,1.219766,1.050390,...,1.084482,1.374359,1.421880,1.105222,1.296508,1.476123,1.286649,1.290494,1.330722,1.268026
60979,FOODS_3_827_WI_3_evaluation,1.785979,1.645527,1.535277,1.591472,1.951546,2.090621,1.987258,1.982190,1.675242,...,1.735642,1.953573,1.788638,1.488734,1.819079,1.767774,1.622244,1.902803,2.010796,1.995631


In [26]:
final_sub.describe()

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
count,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,...,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000,60980.000000
mean,1.292027,1.196487,1.172027,1.181058,1.405504,1.658672,1.711636,1.454986,1.366627,1.371566,...,1.490088,1.853698,1.963374,1.452785,1.345249,1.352967,1.293922,1.468637,1.738965,1.714385
std,2.975136,2.743690,2.697842,2.713226,3.307656,3.807698,3.759851,3.278799,3.144729,3.096615,...,3.458639,4.270506,4.441997,3.209841,3.074470,3.078577,2.925378,3.434469,4.025350,3.974327
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.062425,0.000000,0.000000,0.000000,...,0.000000,0.052599,0.098382,0.000000,0.000000,0.000000,0.000000,0.000000,0.029722,0.071428
50%,0.480513,0.445251,0.435754,0.432826,0.559800,0.745420,0.806520,0.602949,0.534388,0.554447,...,0.604862,0.870727,0.955764,0.604922,0.532321,0.538321,0.508019,0.609051,0.788709,0.773960
75%,1.246043,1.092807,1.068641,1.064947,1.405001,1.885322,2.000000,1.505888,1.300990,1.356834,...,1.516852,2.000000,2.000000,1.516940,1.300522,1.328280,1.221135,1.490061,2.000000,2.000000
max,133.000000,117.000000,113.000000,100.497294,196.000000,155.071575,150.637701,118.532790,143.000000,107.000000,...,143.000000,163.035064,187.000000,118.663332,112.275910,111.525633,110.529054,145.478205,174.391008,152.465622


In [27]:
final_sub.to_csv('lgb_bystore_final4.csv',index=False)